In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
data = pd.read_csv('iris.csv', header=None)
X = data.iloc[: , :4]
y =pd.Series(map(str, data.iloc[: , 4]))

In [12]:
from sklearn import tree




In [13]:
clf = tree.DecisionTreeClassifier(criterion='gini')



In [117]:

import subprocess
import statistics
import warnings

warnings.filterwarnings("ignore")

# Define a list of input values
n = 25
# Initialize an empty list to store the outputs
outputs = []

# Loop over the input values and execute the command
for i in range(n):
    # Execute the command and capture the output
    output = subprocess.check_output(
        ['cargo run --release'], shell=True, text=True)

    # Append the output to the list of outputs
    outputs.append(float(output.strip('ms\n')))
    # outputs.append(output)


# Print the outputs


 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

...

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

 --> src/main.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/main.rs:5:17
  |
5 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

 --> src/dtree.rs:6:17
  |
6 | use std::time::{Duration, Instant};
  |                 ^^^^^^^^

 --> src/dtree.rs:7:5
  |
7 | use array_tool::vec::Uniq;
  |     ^^^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:1:5
  |
1 | use std::fs::File;
  |     ^^^^^^^^^^^^^

 --> src/dtree/utils.rs:2:5
  |
2 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree/utils.rs:4:5
  |
4 | use csv::Error;
  |     ^^^^^^^^^^

   --> src/dtree.rs:214:9
    |
214 |         (1.0 - gini)
 

In [124]:
t = %timeit -o -r 5 -n 5 clf.fit(X, y)

print(f"{sum(outputs)/n} ms ± {statistics.stdev(outputs)} ms")

print(f"difference: {(sum(outputs)/n)/(np.mean(t.all_runs)/5*1000)}")

15.1 ms ± 1.31 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
37.581105279999996 ms ± 3.1141093023407254 ms
difference: 2.4890804233793005
